In [1]:
import pandas as pd


In [8]:
raw_data_path = 'data/raw.csv' 
raw_data = pd.read_csv(raw_data_path)
raw_data.head()

,GID,YORF,NAME,GWEIGHT,G0.05,G0.1,G0.15,G0.2,G0.25,G0.3,...,L0.15,L0.2,L0.25,L0.3,U0.05,U0.1,U0.15,U0.2,U0.25,U0.3
0,GENE1331X,A_06_P5820,SFB2 || ER to Golgi transport || molecul...,1,-0.24,-0.13,-0.21,-0.15,-0.05,-0.05,...,0.13,0.20,0.17,0.11,-0.06,-0.26,-0.05,-0.28,-0.19,0.09
1,GENE4924X,A_06_P5866,|| biological process unknown || mol...,1,0.28,0.13,-0.40,-0.48,-0.11,0.17,...,0.02,0.04,0.03,0.01,-1.02,-0.91,-0.59,-0.61,-0.17,0.18
2,GENE4690X,A_06_P1834,QRI7 || proteolysis and peptidolysis || ...,1,-0.02,-0.27,-0.27,-0.02,0.24,0.25,...,-0.07,-0.05,-0.13,-0.04,-0.91,-0.94,-0.42,-0.36,-0.49,-0.47
3,GENE1177X,A_06_P4928,CFT2 || mRNA polyadenylylation* || RNA b...,1,-0.33,-0.41,-0.24,-0.03,-0.03,0.00,...,-0.05,0.02,0.00,0.08,-0.53,-0.51,-0.26,0.05,-0.14,-0.01
4,GENE511X,A_06_P5620,SSO2 || vesicle fusion* || t-SNARE activ...,1,0.05,0.02,0.40,0.34,-0.13,-0.14,...,0.00,-0.11,0.04,0.01,-0.45,-0.09,-0.13,0.02,-0.09,-0.03


In [11]:
import pandas as pd

# Function to separate the NAME column into multiple columns
def separate_name_column(df):
    # Splitting the 'NAME' column
    name_split = df['NAME'].str.split(' \|\| ', expand=True)

    # Selecting only the first four columns in case there are more than four
    name_split = name_split.iloc[:, :4]

    # Naming the new columns
    name_split.columns = ['gene_name', 'biological_process', 'molecular_function', 'systematic_name']

    # Adding the new columns to the original dataframe
    df = pd.concat([df, name_split], axis=1)
    df.drop('NAME', axis=1, inplace=True)
    
    return df

# Function to pivot the data
def pivot_data(df):
    pivot_cols = [col for col in df.columns if col.startswith(('G', 'L', 'N', 'P', 'S', 'U'))]
    id_vars = [col for col in df.columns if col not in pivot_cols]
    df_melted = df.melt(id_vars=id_vars, value_vars=pivot_cols, var_name='nutrient_rate', value_name='expression')
    df_melted['nutrient'] = df_melted['nutrient_rate'].str.extract(r'([GLNPSU])')[0]
    df_melted['rate'] = df_melted['nutrient_rate'].str.extract(r'([\d.]+)')[0]
    df_melted.drop('nutrient_rate', axis=1, inplace=True)
    return df_melted

# Applying the functions to the raw data
raw_data = pd.read_csv('data/raw.csv')  # Replace with the correct file path
separated_data = separate_name_column(raw_data)
tidy_data = pivot_data(separated_data)

# Saving the tidy data to a CSV file
tidy_data.to_csv('data/cleaned_data.csv', index=False)

# Displaying the first few rows of the tidy data
tidy_data.head()


,YORF,gene_name,biological_process,molecular_function,systematic_name,expression,nutrient,rate
0,A_06_P5820,SFB2,ER to Golgi transport,molecular function unknown,YNL049C,GENE1331X,G,NaN
1,A_06_P5866,,biological process unknown,molecular function unknown,YNL095C,GENE4924X,G,NaN
2,A_06_P1834,QRI7,proteolysis and peptidolysis,metalloendopeptidase activity,YDL104C,GENE4690X,G,NaN
3,A_06_P4928,CFT2,mRNA polyadenylylation*,RNA binding,YLR115W,GENE1177X,G,NaN
4,A_06_P5620,SSO2,vesicle fusion*,t-SNARE activity,YMR183C,GENE511X,G,NaN
